In [166]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [167]:
# url = "https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=action&sort=boxoffice_gross_us,desc"
# url2 = "https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=action&sort=boxoffice_gross_us,desc&start=51&ref_=adv_nxt"
# response = requests.get(url2).content
# soup = BeautifulSoup(response, "html.parser")

# Define Constants
base_url = "https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres="
appendix = "&sort=boxoffice_gross_us,desc"
appendix2 = "&start="
appendix3 = "&ref_=adv_nxt"
start_nums = ["51","101","151","201",]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [168]:
def scrape_page(this_url:str):
    """Function accepts a url as a string and returns a pandas DataFrame containing one page of movie data"""
    
    r = requests.get(this_url).content
    s = BeautifulSoup(r, "html.parser")
    
    content_boxes = s.find_all("div", {"class":"lister-item-content"})

    titles_list =[]
    years_list = []
    certificates_list=[]
    genres_list=[]
    imdb_ratings_list=[]
    votes_list=[]
    gross_list=[]

    for b in content_boxes:
        title = b.find("a").text
        year = b.find("span", {"class":"lister-item-year text-muted unbold"}).text
        certificate = b.find("span", {"class": "certificate"}).text
        genre = b.find("span", {"class": "genre"}).text
        imdb_rating = b.find("div", {"class":"inline-block ratings-imdb-rating"}).text
        votes = b.find_all("span", {"name":"nv"})[0].text
        gross = b.find_all("span", {"name":"nv"})[1].text

        titles_list.append(title)
        years_list.append(year)
        certificates_list.append(certificate)
        genres_list.append(genre)
        imdb_ratings_list.append(imdb_rating)
        votes_list.append(votes)
        gross_list.append(gross)
        
    data = pd.DataFrame({"Title": titles_list, 
                 "Release Year": years_list,
                 "Certificate": certificates_list,
                "Genres": genres_list,
                "IMDB Rating": imdb_ratings_list,
                "Votes": votes_list,
                "Gross Revenue": gross_list})
        
    return data

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [186]:
def scrape_genre(this_genre):
    """Function accepts a genre from the Genres list and scrapes the top 250 grossing by US box office"""
    frames=[]
    for page in range(5):
        # Generate URL to follow
        if page == 0:
            url = base_url + this_genre + appendix
        else:
            url = base_url + this_genre + appendix + appendix2 + start_nums[page - 1] + appendix3
        # Scrape that page and append - if it cannot scrape the page, skip
        try:   
            page_data = scrape_page(url)
            frames.append(page_data)
        except:
            pass
    # Compile

    full = pd.concat(frames)
    full = full.assign(genre_pull=this_genre)
    return full

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [178]:
def scrape_across_genres(list_of_genres):
    """Function accepts a list of genres and scrapes the top 250 grossing by US box office for each"""
    frames=[]
    for genre in list_of_genres:
        try:
            df=scrape_genre(genre)
            print(f"Successfully Scraped Top 250 Film Data for {genre}")
            frames.append(df)
        except:
            print(f"Failed to Scrape Top 250 Fild Data for {genre}")
    
    return pd.concat(frames)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [181]:
tester_genres = ["comedy", "action", "family"]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [180]:
big_dog = scrape_across_genres(tester_genres)

,Title,Release Year,Certificate,Genres,IMDB Rating,Votes,Gross Revenue,genre_pull
0,Incredibles 2,(2018),PG,"\nAnimation, Action, Adventure",\n\n7.6\n,"280,789",$608.58M,comedy
1,Finding Dory,(2016),PG,"\nAnimation, Adventure, Comedy",\n\n7.3\n,"264,085",$486.30M,comedy
2,Frozen II,(2019),PG,"\nAnimation, Adventure, Comedy",\n\n6.8\n,"161,827",$477.37M,comedy
3,Shrek 2,(2004),PG,"\nAnimation, Adventure, Comedy",\n\n7.3\n,"447,215",$436.47M,comedy
4,Toy Story 4,(2019),G,"\nAnimation, Adventure, Comedy",\n\n7.7\n,"234,294",$434.04M,comedy
...,...,...,...,...,...,...,...,...
45,Alice Through the Looking Glass,(2016),PG,"\nAdventure, Family, Fantasy",\n\n6.2\n,"103,779",$77.04M,family
46,The Muppet Movie,(1979),G,"\nAdventure, Comedy, Family",\n\n7.6\n,"35,424",$76.66M,family
47,Father of the Bride Part II,(1995),PG,"\nComedy, Family, Romance",\n\n6.0\n,"37,348",$76.59M,family
48,Fantasia,(1940),G,"\nAnimation, Family, Fantasy",\n\n7.7\n,"93,957",$76.41M,family


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [192]:
genres = ["comedy",
          "family", 
          "talk-show",
          "romance",
          "fantasy",
          "action", 
          "game-show",
          "musical", # fixed
          "horror",
          "documentary",
          "history", # fixed
          "western", # fixed
          "film-noir", # fixed
          "drama", 
          "short",
          "animation",
          "adventure",
          "music", # fixed
          "sci-fi",
          "crime", 
          "news", 
          "mystery",
          "thriller",
          "sport", # fixed
          "biography",
          "war"] # fixed

# excluded reality-tv and adult because they both had zero results
  
full_pull = scrape_across_genres(genres)

Successfully Scraped Top 250 Film Data for comedy
Successfully Scraped Top 250 Film Data for family
Successfully Scraped Top 250 Film Data for talk-show
Successfully Scraped Top 250 Film Data for romance
Successfully Scraped Top 250 Film Data for fantasy
Successfully Scraped Top 250 Film Data for action
Successfully Scraped Top 250 Film Data for game-show
Successfully Scraped Top 250 Film Data for musical
Successfully Scraped Top 250 Film Data for horror
Successfully Scraped Top 250 Film Data for documentary
Successfully Scraped Top 250 Film Data for history
Successfully Scraped Top 250 Film Data for western
Successfully Scraped Top 250 Film Data for film-noir
Successfully Scraped Top 250 Film Data for drama
Successfully Scraped Top 250 Film Data for short
Successfully Scraped Top 250 Film Data for animation
Successfully Scraped Top 250 Film Data for adventure
Successfully Scraped Top 250 Film Data for music
Successfully Scraped Top 250 Film Data for sci-fi
Successfully Scraped Top 250

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [193]:
full_pull.to_csv("tester_imdb_scrape.csv", index=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [198]:
len(full_pull["Title"].unique())

2091

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [115]:
# This is archived code - not used

content_boxes = soup.find_all("div", {"class":"lister-item-content"})

titles_list =[]
years_list = []
certificates_list=[]
genres_list=[]
imdb_ratings_list=[]
votes_list=[]
gross_list=[]

for b in content_boxes:
    title = b.find("a").text
    year = b.find("span", {"class":"lister-item-year text-muted unbold"}).text
    certificate = b.find("span", {"class": "certificate"}).text
    genre = b.find("span", {"class": "genre"}).text
    imdb_rating = b.find("div", {"class":"inline-block ratings-imdb-rating"}).text
    votes = b.find_all("span", {"name":"nv"})[0].text
    gross = b.find_all("span", {"name":"nv"})[1].text
    
    titles_list.append(title)
    years_list.append(year)
    certificates_list.append(certificate)
    genres_list.append(genre)
    imdb_ratings_list.append(imdb_rating)
    votes_list.append(votes)
    gross_list.append(gross)

In [116]:
data = pd.DataFrame({"Title": titles_list, 
                     "Release Year": years_list,
                     "Certificate": certificates_list,
                    "Genres": genres_list,
                    "IMDB Rating": imdb_ratings_list,
                    "Votes": votes_list,
                    "Gross Revenue": gross_list})
data

,Title,Release Year,Certificate,Genres,IMDB Rating,Votes,Gross Revenue
0,Indiana Jones and the Temple of Doom,(1984),PG,"\nAction, Adventure",\n\n7.5\n,"474,670",$179.87M
1,Top Gun,(1986),PG,"\nAction, Drama",\n\n6.9\n,"319,843",$179.80M
2,Men in Black 3,(2012),PG-13,"\nAction, Adventure, Comedy",\n\n6.8\n,"353,361",$179.02M
3,How to Train Your Dragon 2,(2014),PG,"\nAnimation, Action, Adventure",\n\n7.8\n,"324,549",$177.00M
4,Rise of the Planet of the Apes,(2011),PG-13,"\nAction, Drama, Sci-Fi",\n\n7.6\n,"516,510",$176.76M
5,Captain America: The First Avenger,(2011),PG-13,"\nAction, Adventure, Sci-Fi",\n\n6.9\n,"810,409",$176.65M
6,The Karate Kid,(2010),PG,"\nAction, Drama, Family",\n\n6.2\n,"176,720",$176.59M
7,The Bourne Supremacy,(2004),PG-13,"\nAction, Mystery, Thriller",\n\n7.7\n,"454,596",$176.24M
8,The Lego Batman Movie,(2017),PG,"\nAnimation, Action, Adventure",\n\n7.3\n,"142,703",$175.75M
9,Crocodile Dundee,(1986),PG-13,"\nAction, Adventure, Comedy",\n\n6.6\n,"102,582",$174.64M
